# 注意： 在运行这个程序时， 一定要关闭 Fiddler4 抓包软件
##  真正的 url 应该是 Fiddler 中 Raw 选项卡中的 url 

In [1]:
# 模拟登录古诗文网

# 以后 在 开发的过程中， 只要遇到 Cookie 
#      想都别想 就用 session=requests.Session() 去处理， 不要手动去处理

import requests
from lxml import etree
import urllib

In [ ]:
# 此类 会向 服务器中保存的 User-Agent 中 随机选择一个，
# 此类在实际 执行中会向服务器发送一个网络请求，
# 但是 此服务器 不是很稳定， 故这个 获取 User-Agent 值的 方法 不太好用
# from fake_useragent import UserAgent
# ua = UserAgent(verify_ssl=False,use_cache_server=False).random
# print(ua)


In [2]:
import http.client, mimetypes, urllib, json, time, requests

######################################################################

class YDMHttp:

    apiurl = 'http://api.yundama.com/api.php'
    username = ''
    password = ''
    appid = ''
    appkey = ''

    def __init__(self, username, password, appid, appkey):
        self.username = username  
        self.password = password
        self.appid = str(appid)
        self.appkey = appkey

    def request(self, fields, files=[]):
        response = self.post_url(self.apiurl, fields, files)
        response = json.loads(response)
        return response
    
    def balance(self):
        data = {'method': 'balance', 'username': self.username, 'password': self.password, 'appid': self.appid, 'appkey': self.appkey}
        response = self.request(data)
        if (response):
            if (response['ret'] and response['ret'] < 0):
                return response['ret']
            else:
                return response['balance']
        else:
            return -9001
    
    def login(self):
        data = {'method': 'login', 'username': self.username, 'password': self.password, 'appid': self.appid, 'appkey': self.appkey}
        response = self.request(data)
        if (response):
            if (response['ret'] and response['ret'] < 0):
                return response['ret']
            else:
                return response['uid']
        else:
            return -9001

    def upload(self, filename, codetype, timeout):
        data = {'method': 'upload', 'username': self.username, 'password': self.password, 'appid': self.appid, 'appkey': self.appkey, 'codetype': str(codetype), 'timeout': str(timeout)}
        file = {'file': filename}
        response = self.request(data, file)
        if (response):
            if (response['ret'] and response['ret'] < 0):
                return response['ret']
            else:
                return response['cid']
        else:
            return -9001

    def result(self, cid):
        data = {'method': 'result', 'username': self.username, 'password': self.password, 'appid': self.appid, 'appkey': self.appkey, 'cid': str(cid)}
        response = self.request(data)
        return response and response['text'] or ''

    def decode(self, filename, codetype, timeout):
        cid = self.upload(filename, codetype, timeout)
        if (cid > 0):
            for i in range(0, timeout):
                result = self.result(cid)
                if (result != ''):
                    return cid, result
                else:
                    time.sleep(1)
            return -3003, ''
        else:
            return cid, ''

    def report(self, cid):
        data = {
                    'method': 'report',
                    'username': self.username,
                    'password': self.password,
                    'appid': self.appid,
                    'appkey': self.appkey,
                    'cid': str(cid),
                    'flag': '0'
               }
        response = self.request(data)
        if (response):
            return response['ret']
        else:
            return -9001

    def post_url(self, url, fields, files=[]):
        for key in files:
            files[key] = open(files[key], 'rb');
        res = requests.post(url, files=files, data=fields)
        return res.text

######################################################################


######################################################################


In [3]:
def get_codeData(userName,pwd,codePath,codeType):
    """
        codePath: 图片地址
        codeType: 验证码类型
        
    """
    # 用户名(普通用户)
    username    = userName

    # 密码
    password    = pwd                            

    # 软件ＩＤ，开发者分成必要参数。登录开发者后台【我的软件】获得！
    appid       = 7788                                     

    # 软件密钥，开发者分成必要参数。登录开发者后台【我的软件】获得！
    appkey      = '12caf92090f726e010f41807397b2302'    

    # 图片文件
    filename    = codePath                        

    # 验证码类型，# 例：1004表示4位字母数字，不同类型收费不同。请准确填写，否则影响识别率。
    # 在此查询所有类型 http://www.yundama.com/price.html
    codetype    = codeType

    # 超时时间，秒
    timeout     = 30                                   
    result = None
    # 检查
    if (username == 'username'):
        print('请设置好相关参数再测试')
    else:
        # 初始化
        yundama = YDMHttp(username, password, appid, appkey)

        # 登陆云打码
        uid = yundama.login();
        print('uid: %s' % uid)

        # 查询余额
        balance = yundama.balance();
        print('balance: %s' % balance)

        # 开始识别，图片路径，验证码类型ID，超时时间（秒），识别结果
        cid, result = yundama.decode(filename, codetype, timeout);
        print('cid: %s, result: %s' % (cid, result))
    return result

In [10]:


url = 'https://so.gushiwen.org/user/login.aspx?from=http://so.gushiwen.org/user/collect.aspx'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:66.0) Gecko/20100101 Firefox/66.0',
}

# 爬取登录页面源码 ， 以 获得 验证码的 URL 
response = requests.get(url=url,headers=headers)
pg_text = response.text

# 运用 Xpath 表达式 获得 验证码图片的 URL 
tree = etree.HTML(pg_text)
verify_url ='https://so.gushiwen.org'+ tree.xpath('//*[@id="imgCode"]/@src')[0]
session = requests.Session()
verify_data = session.get(url=verify_url,headers=headers).content

with open('./verify.jpg','wb') as f:
    f.write(verify_data)
# 解析 验证码
code_data = get_codeData('suosuo','718094RMB718094','./verify.jpg',1004)  # 四位英文数字
# 登录 URL , 
submit_url = 'https://so.gushiwen.org/user/login.aspx?from=http%3a%2f%2fso.gushiwen.org%2fuser%2fcollect.aspx'

# __VIEWSTATE 和  __VIEWSTATEGENERATOR 的值 一般放置在 页面源码的 隐藏标签中存储。
data = {
    "__VIEWSTATE": "IttZTgX6+gCZ1FTV7bkt7BeLZd3vSlpVGemhyX3W1PAqtfAjI7PUyhG5NwmU16J7nTkNYsBYHtjqG7XCf037K0OMlJNh00gTMey6dOPsXPoupXOl5VKzaq+dPAA=",
    "__VIEWSTATEGENERATOR": "C93BE1AE",
    "from": "http://so.gushiwen.org/user/collect.aspx",
    "email": "16603312116",
    "pwd": "7180918094",
    "code": code_data,
    "denglu": "登录",
}
#爬取 个人 登录成功的 个人首页
self_text = session.post(url=submit_url,data=data,headers=headers).content

with open('modern.html','wb') as f:
    f.write(self_text)
print('登录成功！')



uid: 73727
balance: 2890
cid: 1874931657, result: h8h8
登录成功！
